In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from pathlib import Path

# Set plotly theme
pio.templates.default = "plotly_white"

# Load the data
data_path = Path("/Users/yeva/imperial/master-proj/gradual-sem-causal-aba/results/gradual/v2_ablation_random_graphs_7nodes/cpdag_metrics.csv")
df = pd.read_csv(data_path)

print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")

Data shape: (850, 30)
Columns: ['nnz', 'fdr', 'tpr', 'fpr', 'precision', 'recall', 'F1', 'shd', 'sid_low', 'sid_high', 'dataset', 'seed', 'n_nodes', 'n_edges', 'neighbourhood_n_nodes', 'max_cycle_length', 'max_ct_depth', 'max_path_length', 'max_c_set_size', 'search_depth', 'elapsed_bsaf_creation', 'elapsed_model_solution', 'is_converged', 'fact_ranking_method', 'model_ranking_method', 'num_edges_est', 'best_model', 'aba_elapsed', 'ranking_elapsed', 'best_I']

First few rows:


In [3]:
df['neighbourhood_n_nodes'] = df['max_cycle_length']
df['use_collider_arguments'] = df['max_ct_depth'] > -1  # Assuming -1 indicates no collider arguments used

In [4]:
# Explore the unique values for ablation parameters
print("Unique values for ablation parameters:")
print(f"neighbourhood_n_nodes: {sorted(df['neighbourhood_n_nodes'].unique())}")
print(f"use_collider_arguments: {sorted(df['use_collider_arguments'].unique())}")
print(f"max_c_set_size: {sorted(df['max_c_set_size'].unique())}")
print(f"search_depth: {sorted(df['search_depth'].unique())}")

# Compute mean and std across runs, grouped by the ablation parameters
groupby_cols = ['neighbourhood_n_nodes', 'use_collider_arguments', 'max_c_set_size', 'search_depth']
metrics_of_interest = ['sid_low', 'sid_high']

# Group by ablation parameters and compute statistics
grouped_stats = df.groupby(groupby_cols)[metrics_of_interest].agg(['mean', 'std', 'count']).reset_index()

# Flatten column names
grouped_stats.columns = [col[0] if col[1] == '' else f'{col[0]}_{col[1]}' for col in grouped_stats.columns]

print(f"\nGrouped statistics shape: {grouped_stats.shape}")
grouped_stats.head(10)

Unique values for ablation parameters:
neighbourhood_n_nodes: [np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)]
use_collider_arguments: [np.False_, np.True_]
max_c_set_size: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
search_depth: [np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]

Grouped statistics shape: (17, 10)


,neighbourhood_n_nodes,use_collider_arguments,max_c_set_size,search_depth,sid_low_mean,sid_low_std,sid_low_count,sid_high_mean,sid_high_std,sid_high_count
0,3,True,5,10,19.36,4.822397,50,21.42,5.522089,50
1,4,True,5,10,17.74,5.009420,50,22.72,6.857262,50
2,5,True,5,10,18.08,5.375075,50,22.48,6.810077,50
3,6,True,5,10,18.02,5.460395,50,22.46,6.842872,50
4,7,False,5,10,18.64,3.685382,50,23.44,5.897630,50
5,7,True,0,10,18.48,5.003019,50,23.28,5.547715,50
6,7,True,1,10,18.16,6.109077,50,22.60,6.948792,50
7,7,True,2,10,18.02,5.460395,50,22.46,6.842872,50
8,7,True,3,10,18.76,5.208157,50,22.70,6.609363,50
9,7,True,4,10,18.76,5.208157,50,22.70,6.609363,50


In [5]:
all(grouped_stats.sid_high_count == 50)

True

In [6]:
def create_ablation_plot2(param_name, fixed_values_dict=None, n_edges=7):
    """
    Create ablation plot for a specific parameter while keeping others fixed.
    Plots both SID-best and SID-worst as two traces on a single plot.
    """
    # Default fixed values (median or reasonable defaults)
    default_fixed = {
        'neighbourhood_n_nodes': 7,
        'use_collider_arguments': True, 
        'max_c_set_size': 5,
        'search_depth': 10
    }
    if fixed_values_dict:
        default_fixed.update(fixed_values_dict)
    varying_param = param_name
    fixed_params = {k: v for k, v in default_fixed.items() if k != varying_param}
    filtered_data = df.copy()
    for param, value in fixed_params.items():
        filtered_data = filtered_data[filtered_data[param] == value]
    if len(filtered_data) == 0:
        print(f"No data found for the specified fixed parameters: {fixed_params}")
        return None
    filtered_data['sid_low'] = filtered_data['sid_low'] / n_edges
    filtered_data['sid_high'] = filtered_data['sid_high'] / n_edges
    param_stats = filtered_data[filtered_data[varying_param] != default_fixed[varying_param]]
    baseline = filtered_data[filtered_data[varying_param] == default_fixed[varying_param]]
    non_varying_params = list(fixed_params.keys())
    param_stats = param_stats.merge(
        baseline[[*non_varying_params, 'seed', 'sid_low', 'sid_high']],
        on=['seed', *non_varying_params], suffixes=('', '_baseline'), how='left'
    )
    param_stats['sid_low'] = 100 * (param_stats['sid_low'] - param_stats['sid_low_baseline']) / param_stats['sid_low_baseline']
    param_stats['sid_high'] = 100 * (param_stats['sid_high'] - param_stats['sid_high_baseline']) / param_stats['sid_high_baseline']
    param_stats = param_stats.groupby(varying_param)[['sid_low', 'sid_high']].agg(['mean', 'std', 'count']).reset_index()
    param_stats.columns = [col[0] if col[1] == '' else f'{col[0]}_{col[1]}' for col in param_stats.columns]

    # Single plot with two traces
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=param_stats[varying_param],
            y=param_stats['sid_low_mean'],
            error_y=dict(type='data', array=param_stats['sid_low_std'], visible=True),
            mode='lines+markers',
            name='SID Best',
            line=dict(width=3),
            marker=dict(symbol='triangle-up-dot', size=12),
            opacity=0.9,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=param_stats[varying_param],
            y=param_stats['sid_high_mean'],
            error_y=dict(type='data', array=param_stats['sid_high_std'], visible=True),
            mode='lines+markers',
            name='SID Worst',
            line=dict(width=3),
            marker=dict(symbol='triangle-down-dot', size=12),
            opacity=0.9,
        )
    )
    mapping = {
        'neighbourhood_n_nodes': 'Neighborhood Nodes',
        'use_collider_arguments': 'Use Collider Arguments',
        'max_c_set_size': 'Max Conditioning Set Size',
        'search_depth': 'Search Depth'
    }
    fig.update_xaxes(title_text=mapping[varying_param].title())
    fig.update_yaxes(title_text='Change Percent (Mean ± Std)', ticksuffix="%")
    fig.write_image(f'{param_name}_ablation.png', scale=3, width=500, height=400)
    fig.show()
    return param_stats

In [7]:
# 1. Neighbourhood N Nodes ablation
print("\n1. Neighbourhood N Nodes Ablation")
stats_cycle = create_ablation_plot2('neighbourhood_n_nodes')

# 3. Max C Set Size ablation  
print("\n3. Max C Set Size Ablation")
stats_c_set = create_ablation_plot2('max_c_set_size')

# 4. Search Depth ablation
print("\n4. Search Depth Ablation") 
stats_search = create_ablation_plot2('search_depth')


1. Neighbourhood N Nodes Ablation



3. Max C Set Size Ablation



4. Search Depth Ablation


In [8]:
def plot_ablation_subplots(param_names, fixed_values_dicts=None, n_edges=7):
    """
    Plot ablation results for multiple parameters as subplots with shared legend.
    param_names: list of parameter names to ablate
    fixed_values_dicts: list of dicts with fixed values for each ablation (or None)
    """
    if fixed_values_dicts is None:
        fixed_values_dicts = [None] * len(param_names)
    mapping = {
        'neighbourhood_n_nodes': 'Neighborhood Nodes',
        'use_collider_arguments': 'Use Collider Arguments',
        'max_c_set_size': 'Max Conditioning Set Size',
        'search_depth': 'Top k'
    }
    fig = make_subplots(rows=1, cols=len(param_names), shared_yaxes=True, horizontal_spacing=0.08)
    for i, (param_name, fixed_values_dict) in enumerate(zip(param_names, fixed_values_dicts)):
        # --- Compute stats as in create_ablation_plot2 ---
        default_fixed = {
            'neighbourhood_n_nodes': 7,
            'use_collider_arguments': True, 
            'max_c_set_size': 5,
            'search_depth': 10
        }
        if fixed_values_dict:
            default_fixed.update(fixed_values_dict)
        varying_param = param_name
        fixed_params = {k: v for k, v in default_fixed.items() if k != varying_param}
        filtered_data = df.copy()
        for param, value in fixed_params.items():
            filtered_data = filtered_data[filtered_data[param] == value]
        if len(filtered_data) == 0:
            print(f"No data found for the specified fixed parameters: {fixed_params}")
            continue
        filtered_data['sid_low'] = filtered_data['sid_low'] / n_edges
        filtered_data['sid_high'] = filtered_data['sid_high'] / n_edges
        param_stats = filtered_data[filtered_data[varying_param] != default_fixed[varying_param]]
        baseline = filtered_data[filtered_data[varying_param] == default_fixed[varying_param]]
        non_varying_params = list(fixed_params.keys())
        param_stats = param_stats.merge(
            baseline[[*non_varying_params, 'seed', 'sid_low', 'sid_high']],
            on=['seed', *non_varying_params], suffixes=('', '_baseline'), how='left'
        )
        param_stats['sid_low'] = 100 * (param_stats['sid_low'] - param_stats['sid_low_baseline']) / param_stats['sid_low_baseline']
        param_stats['sid_high'] = 100 * (param_stats['sid_high'] - param_stats['sid_high_baseline']) / param_stats['sid_high_baseline']
        param_stats = param_stats.groupby(varying_param)[['sid_low', 'sid_high']].agg(['mean', 'std', 'count']).reset_index()
        param_stats.columns = [col[0] if col[1] == '' else f'{col[0]}_{col[1]}' for col in param_stats.columns]
        # --- Add traces ---
        fig.add_trace(
            go.Scatter(
                x=param_stats[varying_param],
                y=param_stats['sid_low_mean'],
                error_y=dict(type='data', array=param_stats['sid_low_std'], visible=True),
                mode='lines+markers',
                name='SID Best' if i == 0 else None,
                line=dict(width=3, color='#1f77b4'),
                marker=dict(symbol='triangle-up-dot', size=12, color='#1f77b4'),
                opacity=0.9,
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )
        fig.add_trace(
            go.Scatter(
                x=param_stats[varying_param],
                y=param_stats['sid_high_mean'],
                error_y=dict(type='data', array=param_stats['sid_high_std'], visible=True),
                mode='lines+markers',
                name='SID Worst' if i == 0 else None,
                line=dict(width=3, color='#ff7f0e'),
                marker=dict(symbol='triangle-down-dot', size=12, color='#ff7f0e'),
                opacity=0.9,
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )
        fig.update_xaxes(title_text=mapping[varying_param], row=1, col=i+1)
    fig.update_yaxes(title_text='Change Percent (Mean ± Std)', ticksuffix="%", row=1, col=1)
    fig.update_layout(
        width=300*len(param_names)+50, height=350,
        legend=dict(orientation="h", xanchor="center", x=0.5, yanchor="bottom", y=1.08),
        template='plotly_white',
        margin=dict(l=30, r=30, b=60, t=30)
    )
    fig.write_image('ablation_subplots.png', scale=3, width=300*len(param_names)+50, height=350)
    fig.show()

# Example usage:
plot_ablation_subplots(
    ['neighbourhood_n_nodes', 'max_c_set_size', 'search_depth']
)